# **DIMEMEX — Complete Project Pipeline Notebook**

*Multilingual Meme Translation & Hate Speech Analysis*

---

## **Members**

* **Bárbara** (Text)
* **Amanda** (Text + Description)
* **Juan David Nieto** (Text + Description + Image)
* **Luisa** (Image)

---

## **Project Overview**

This project analyzes whether **offensive or hate speech content in memes is preserved after translating the memes from Spanish to Portuguese**.

We work with the **DIMEMEX** dataset, which contains:

* Meme **text**
* Meme **description**
* Meme **image**
* Labels: *hate speech*, *inappropriate content*, *neither*

This project has **two main tasks**:

### **Task 1 — Translation Quality Evaluation**

Translate the Spanish text to Portuguese and evaluate translation quality using standard NLP metrics:

* **BLEURT**
* **BERTScore**
* **COMET-Kiwi**
* **chrF**

### **Task 2 — Hate Speech Detection (Multimodal Fine-Tuning)**

Fine-tune models to classify:

* Hate speech
* Inappropriate content
* Neither

We fine-tune under four input settings:

1. Text
2. Text + Description
3. Text + Description + Image (Multimodal)
4. Image

---

## **📌 Objectives**

1. Evaluate whether offensive content is **maintained or lost** during translation.
2. Compare performance of **original Spanish memes vs. translated Portuguese memes**.
3. Train and evaluate multimodal detectors to classify hate speech.
4. Analyze cases where the label changes across languages.
5. Perform **human qualitative analysis** on inconsistent cases.



# Imports

# 🛠️ 1. Install Dependencies

In [2]:
!pip install -q transformers accelerate datasets peft bitsandbytes torch torchvision pillow tqdm scikit-learn matplotlib evaluate
!pip install -q trl # Install TRL for SFTTrainer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 13.5 MB/s eta 0:00:00


In [3]:
import os
import zipfile
import json
import warnings

import pandas as pd
import numpy as np
from PIL import Image

from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

import torch

from datasets import Dataset
import evaluate
from evaluate import load

# Hugging Face - Transformers (Models, Processors, Configs)
from transformers import (
    AutoTokenizer,
    AutoProcessor,
    AutoModelForCausalLM,
    Idefics3ForConditionalGeneration,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)

# Hugging Face - PEFT (LoRA)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)

# Hugging Face - TRL (Training)
from trl import SFTTrainer

from tqdm import tqdm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# 🗂️ 2. Upload Data

In [ ]:
from google.colab import files
import zipfile, os

print("📁 Upload your CSV file...")
print("   (Must contain columns: 'image_path', 'label', 'text', 'description')")

try:
    uploaded = files.upload()
    csv_name = list(uploaded.keys())[0]
except Exception as e:
    print(f"Error or interruption during CSV upload: {e}")
    csv_name = None

print("\n📦 Now upload the ZIP file with all images (e.g., DIMEMEX_images.zip)...")
try:
    uploaded2 = files.upload()
    zip_name = list(uploaded2.keys())[0]
except Exception as e:
    print(f"Error or interruption during ZIP upload: {e}")
    zip_name = None

# Unzip images
IMAGE_ROOT = "DIMEMEX_images"
if zip_name:
    os.makedirs(IMAGE_ROOT, exist_ok=True)
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(IMAGE_ROOT)
    print("✅ Files uploaded and extracted.")
else:
    print("⚠️ ZIP file upload skipped or failed.")

# ⚙️ 3. Dataset Configuration and Analysis

In [ ]:
import pandas as pd
from PIL import Image

if csv_name:
    DATA_CSV = csv_name
    df = pd.read_csv(DATA_CSV)

    # Verify required columns for multimodal input
    required_cols = ['image_path', 'label', 'text', 'description']
    if not all(col in df.columns for col in required_cols):
        print(f"❌ ERROR: CSV must contain {required_cols}")
    else:
        print("✅ CSV loaded successfully.")

        # Update image paths to point to the extracted folder
        df["image_path"] = df["image_path"].apply(
            lambda p: os.path.join(IMAGE_ROOT, os.path.basename(p))
        )

        # Filter out rows with missing images
        df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)

        print("\n📊 Total dataset size (with valid images):", len(df))
        print(df["label"].value_counts().rename_axis("Label").reset_index(name="Count"))
else:
    print("❌ No CSV file loaded. Cannot proceed.")

# ⚙️ 4. Main Configurations

In [ ]:
# --- Model & Training Params ---
MODEL_ID = "HuggingFaceTB/SmolVLM-256M-Instruct"
BATCH_SIZE = 2
EPOCHS = 2
LR = 2e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Label Mapping ---
# We need this for the final evaluation
labels = df['label'].unique()
label_to_id = {label: i for i, label in enumerate(labels)}
id_to_label = {i: label for i, label in enumerate(labels)}
print(f"Labels mapped: {label_to_id}")

#📦 5. Prepare Train, Validation, and Test Datasets

In [ ]:
# We use the 'label' column to stratify the splits
train_df, test_df = train_test_split(
    df, test_size=0.15, stratify=df["label"], random_state=42
)
train_df, val_df = train_test_split(
    train_df, test_size=0.1, stratify=train_df["label"], random_state=42
)

ds_train = Dataset.from_pandas(train_df.reset_index(drop=True))
ds_val = Dataset.from_pandas(val_df.reset_index(drop=True))
ds_test = Dataset.from_pandas(test_df.reset_index(drop=True))

print(f"Train: {len(ds_train)} | Validation: {len(ds_val)} | Test: {len(ds_test)}")

#🧩 6. Processor and Pre-processing **(Text + Desc + Image)**

In [ ]:
processor = AutoProcessor.from_pretrained(MODEL_ID)

def format_for_sft(example):
    """
    Formats the example for the SmolVLM's Instruction (SFT) format.
    Question: 'Analyze the image, text, and description. What is the category?'
    Answer (Label): 'hate speech' / 'inappropriate content' / 'neither'
    """
    # Load image
    try:
        image = Image.open(example['image_path']).convert("RGB")
    except Exception:
        image = Image.new('RGB', (100, 100), color = 'gray') # Placeholder

    # Construct the Multimodal Prompt
    prompt_text = (
        f"Analyze the following meme components and classify it. "
        f"Text: {example['text']}\n"
        f"Description: {example['description']}\n\n"
        f"What is the category? "
    )

    # SFT format: user message (image + text) and assistant response (label)
    messages = [
        {"role": "user", "content": [image, prompt_text]},
        {"role": "assistant", "content": example['label']}
    ]

    # Use the processor to apply the chat template
    example['text'] = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return example

# Apply the formatting
# This creates a single 'text' column that SFTTrainer will use
ds_train = ds_train.map(format_for_sft, remove_columns=ds_train.column_names)
ds_val = ds_val.map(format_for_sft, remove_columns=ds_val.column_names)
# We apply this to the test set later, before evaluation

# 🧠 7. Model + LoRA

In [ ]:
# 4-bit Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = Idefics3ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

base_model = prepare_model_for_kbit_training(base_model)

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    # Target modules updated for Idefics3/SmolVLM
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

# 🧮 8. Training Configuration

In [ ]:
OUTPUT_DIR = "./SmolVLM_256M_Instruct_DIMEMEX_lora"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=False, # Must be False for 4-bit
    bf16=True,  # Use bf16 for 4-bit
    load_best_model_at_end=True,
    logging_steps=50,
    save_total_limit=3,
    report_to="none", # Disable wandb/tensorboard logging
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=processor, # The processor handles tokenization
    dataset_text_field="text", # The column we created in Section 6
    max_seq_length=1024,
)

# 10. Train Model

In [ ]:
trainer.train()
print("Fine-tuning complete.")

# 11. Loss Curves

In [ ]:
log_file = os.path.join(training_args.output_dir, "checkpoint-XXX", "trainer_state.json")
# Note: You may need to update 'checkpoint-XXX' to the latest checkpoint folder
# For simplicity, we'll access the trainer's state directly.

logs = trainer.state.log_history

train_steps = [x["step"] for x in logs if "loss" in x]
train_loss = [x["loss"] for x in logs if "loss" in x]

eval_steps = [x["step"] for x in logs if "eval_loss" in x]
eval_loss = [x["eval_loss"] for x in logs if "eval_loss" in x]

plt.figure(figsize=(10, 5))
plt.plot(train_steps, train_loss, label="Train Loss")
plt.plot(eval_steps, eval_loss, label="Eval Loss", marker='o')
plt.title("Training and Evaluation Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend()
plt.show()

# ✅ 12. Final Evaluation and Metrics Summary

In [ ]:
# Use the fine-tuned model (already loaded in 'trainer.model')
model = trainer.model
test_predictions = []
test_ground_truth = []

# Loop through the test set
print("Running final evaluation on test set...")
for example in tqdm(ds_test):
    # Re-create the prompt, but without the answer
    prompt_text = (
        f"Analyze the following meme components and classify it. "
        f"Text: {example['text']}\n"
        f"Description: {example['description']}\n\n"
        f"What is the category? "
    )
    image = Image.open(example['image_path']).convert("RGB")
    messages = [{"role": "user", "content": [image, prompt_text]}]

    # Prepare input
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

    # Generate response
    output = model.generate(**inputs, max_new_tokens=20, do_sample=False)
    generated_text = processor.decode(output[0], skip_special_tokens=True)

    # Extract just the answer (the label)
    prediction = generated_text.split("assistant\n")[-1].strip()

    test_predictions.append(prediction)
    test_ground_truth.append(example['label'])

# --- Compute Metrics ---
# Filter out any predictions that were not valid labels
valid_preds = [p for p in test_predictions if p in label_to_id]
valid_gt = [g for p, g in zip(test_predictions, test_ground_truth) if p in label_to_id]

print(f"\n{len(valid_preds)} / {len(test_predictions)} predictions were valid labels.")

print("\n📊 Final Test Set Results:")
print(classification_report(valid_gt, valid_preds, target_names=label_to_id.keys()))

# Manual calculation for overall results
results = {}
results["accuracy"] = accuracy_score(valid_gt, valid_preds)
results["f1_weighted"] = f1_score(valid_gt, valid_preds, average="weighted")
results["precision_weighted"] = precision_score(valid_gt, valid_preds, average="weighted", zero_division=0)
results["recall_weighted"] = recall_score(valid_gt, valid_preds, average="weighted", zero_division=0)

print("\n📊 Summary Metrics:")
for k, v in results.items():
    print(f"{k:20s}: {v:.4f}")

# 💾 13. Save Final Model

In [ ]:
# Save the LoRA adapters
model.save_pretrained(OUTPUT_DIR)
# Save the processor
processor.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Fine-tuning complete. Model saved to: {OUTPUT_DIR}")